In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CONFIGURACIÓN VISUAL (PROFESIONAL) ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# --- FUNCIÓN DE BÚSQUEDA INTELIGENTE ---
def buscar_archivo_por_palabra_clave(palabra_clave, extension=".csv"):
    """
    Busca en la carpeta actual cualquier archivo que contenga 
    la 'palabra_clave' en su nombre (ej: 'clientes').
    """
    directorio_actual = os.getcwd()
    archivos_en_carpeta = os.listdir(directorio_actual)
    
    # Filtramos archivos que contengan la palabra clave y terminen en .csv
    candidatos = [f for f in archivos_en_carpeta if palabra_clave in f and f.endswith(extension)]
    
    if candidatos:
        # Retornamos el primero que encuentre (el más probable)
        print(f"   ✅ Archivo encontrado para '{palabra_clave}': {candidatos[0]}")
        return candidates[0]
    else:
        return None

# --- CARGA Y PROCESAMIENTO (ETL) ---
def cargar_datos_sprint_3():
    print("🚀 SPRINT 3: Iniciando carga de datos...")
    print(f"📂 Buscando en carpeta: {os.getcwd()}\n")
    
    # Diccionario para guardar los dataframes
    dfs = {}
    palabras_clave = ['clientes', 'detalle', 'productos', 'ventas']
    
    try:
        for clave in palabras_clave:
            archivo = buscar_archivo_por_palabra_clave(clave)
            
            if archivo:
                # Usamos 'latin-1' para corregir errores de tildes/ñ
                dfs[clave] = pd.read_csv(archivo, encoding='latin-1')
            else:
                # Error específico si falta un archivo
                raise FileNotFoundError(f"❌ No se encontró ningún archivo que contenga la palabra '{clave}' en esta carpeta.")

        # Asignamos variables
        df_cli = dfs['clientes']
        df_det = dfs['detalle'] # Busca 'detalle' (coincide con 'detalle_ventas...')
        df_prod = dfs['productos']
        df_ven = dfs['ventas']

        # Conversión de fechas
        df_ven['fecha'] = pd.to_datetime(df_ven['fecha'])
        
        print("\n🔄 Fusionando tablas (Data Merging)...")
        
        # 1. Unir Detalle con Ventas (Inner Join)
        df_master = pd.merge(df_det, df_ven, on='id_venta', how='inner')
        
        # 2. Agregar información de Productos (Left Join)
        # Usamos suffixes para que 'nombre_producto' no choque con columnas duplicadas
        df_master = pd.merge(df_master, df_prod, on='id_producto', how='left', suffixes=('', '_prod_dup'))
        
        # 3. Agregar información de Clientes (Left Join)
        df_master = pd.merge(df_master, df_cli, on='id_cliente', how='left', suffixes=('', '_cli_dup'))
        
        print(f"✅ Dataset Maestro generado con éxito: {df_master.shape[0]} registros procesados.")
        return df_master

    except Exception as e:
        print(f"\n⛔ ERROR CRÍTICO DE EJECUCIÓN: {e}")
        print("💡 TIP: Verifica que tus 4 archivos .csv estén en la misma carpeta que este Notebook.")
        return None

# --- ANÁLISIS SPRINT 3 ---

def analisis_categorias_rentables(df):
    """ Tarea 1: ¿Qué categoría genera más ingresos? """
    print("\n📦 Visualizando: Rentabilidad por Categoría...")
    
    # Agrupamos por categoría y sumamos importe
    data = df.groupby('categoria')['importe'].sum().sort_values(ascending=False)
    
    plt.figure()
    ax = sns.barplot(x=data.index, y=data.values, palette='viridis', hue=data.index, legend=False)
    
    plt.title('Ingresos Totales por Categoría', fontweight='bold')
    plt.ylabel('Facturación ($)')
    plt.xlabel('Categoría')
    
    # Formato de dinero en el eje Y
    ax.yaxis.set_major_formatter('${x:,.0f}')
    plt.show()

def analisis_medios_pago(df):
    """ Tarea 2: Preferencia de Medios de Pago (Donut Chart) """
    print("\n💳 Visualizando: Preferencias de Pago...")
    
    # IMPORTANTE: Eliminamos duplicados de id_venta para contar transacciones únicas, no productos
    transacciones_unicas = df.drop_duplicates(subset='id_venta')
    data = transacciones_unicas['medio_pago'].value_counts()
    
    plt.figure(figsize=(8, 8))
    # Colores suaves
    colors = sns.color_palette('pastel')[0:5]
    
    plt.pie(data, labels=data.index, autopct='%1.1f%%', startangle=140, colors=colors, pctdistance=0.85)
    
    # Círculo blanco para efecto Donut
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    plt.title('Distribución de Medios de Pago', fontweight='bold')
    plt.show()

def analisis_geografico(df):
    """ Tarea 3: Ventas por Ciudad """
    print("\n📍 Visualizando: Desempeño Geográfico...")
    
    data = df.groupby('ciudad')['importe'].sum().sort_values(ascending=False)
    
    plt.figure()
    sns.barplot(x=data.values, y=data.index, palette='coolwarm', hue=data.index, legend=False)
    
    plt.title('Ranking de Ventas por Ciudad', fontweight='bold')
    plt.xlabel('Total Vendido ($)')
    plt.ylabel('')
    plt.show()

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    df_sprint3 = cargar_datos_sprint_3()
    
    if df_sprint3 is not None:
        analisis_categorias_rentables(df_sprint3)
        analisis_medios_pago(df_sprint3)
        analisis_geografico(df_sprint3)

🚀 SPRINT 3: Iniciando carga inteligente de datos...
   ❌ ERROR CRÍTICO: No se encontró el archivo para 'clientes'.
